In [1]:
%matplotlib qt5
from process_files import *
from func_flats import *

qt.qpa.xcb: X server does not support XInput 2


In [2]:
config = configobj.ConfigObj('config.ini')
line = config['line']
ffdir = config['flats']['directory']
settings = [f for f in os.listdir(ffdir) if 'settings' in f]
settings = ffdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)

# Other numbers
iline = get_line_num(settings, line)
linestr = 'Line_' + str(iline)
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
wavscan_range = float(settings[linestr]['WaveScanRange'])
wavelength = float(settings[linestr]['LineWavelength'])
wavestep = wavscan_range/(nwav-1)
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(ffdir+os.sep+line))//3

qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [3]:
def process_flat_fit(line, channel, mod):
    # Read the settings
    config = configobj.ConfigObj('config.ini')
    ffdir = config['flats']['directory']
    settings = [f for f in os.listdir(ffdir) if 'settings' in f]
    settings = ffdir + os.sep + settings[0]
    settings = configobj.ConfigObj(settings)

    # Other numbers
    iline = 0
    linestr = 'Line_' + str(get_line_num(settings, line, iline))
    nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
    nwav = int(settings[linestr]['NWavePoints'])
    filtstr = settings[linestr]['Filter']
    modstr = settings[linestr]['Polarimeter\\Modulation']
    nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
    nfpc = nmod*nacc*nwav
    nfpw = nmod*nacc
    ncyc = len(os.listdir(ffdir+os.sep+line))//3
    # Read the data
    ffname = config['flats'][line+'/'+channel]
    ff = np.float64(pf.open(ffname)[0].data)
    print(time_now(), 'reading average flat files: ', '\n', ffname)
    Y, X = ff.shape[0:2]
    ffm = ff[:,:,mod,:]
    
    # Fit the data
    guess_params, fit_params = fit_lines_3d(ffm)
    
    # Save the data
    ffit_name = ffname.replace('_fa', '_mod'+str(mod)+'_ft')
    hdu1 = pf.PrimaryHDU(fit_params)
    hdul = pf.HDUList([hdu1])
    hdul.writeto(ffit_name, overwrite=True)
    hdul.close()
    config['flats'][line+'/'+channel+'_mod'+str(mod)+'_fit'] = ffit_name
    config.write()
    print(time_now(), 'fit data written to: ', '\n', ffit_name)
    return

In [4]:
line = 'Fe_I_6173'
pool = mp.Pool(8)
mp_args = []
mp_args.append([line, 'pol1', 0])
mp_args.append([line, 'pol1', 1])
mp_args.append([line, 'pol1', 2])
mp_args.append([line, 'pol1', 3])
mp_args.append([line, 'pol2', 0])
mp_args.append([line, 'pol2', 1])
mp_args.append([line, 'pol2', 2])
mp_args.append([line, 'pol2', 3])
pool.starmap(process_flat_fit, mp_args)

09:35:27 reading average flat files: 09:35:2709:35:27  
 reading average flat files:  reading average flat files: /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol2_20230502_Flats_1309_Fe_I_6173_fa.FITS  


  /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol2_20230502_Flats_1309_Fe_I_6173_fa.FITS/arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol2_20230502_Flats_1309_Fe_I_6173_fa.FITS



  0%|          | 0/1280 [00:00<?, ?it/s]

09:35:2709:35:27  reading average flat files:  reading average flat files: 09:35:27
 09:35:27  
 /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol1_20230502_Flats_1309_Fe_I_6173_fa.FITSreading average flat files:  reading average flat files: /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol2_20230502_Flats_1309_Fe_I_6173_fa.FITS 
09:35:27 

 
  /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol1_20230502_Flats_1309_Fe_I_6173_fa.FITS

  0%|          | 0/1280 [00:00<?, ?it/s]


/arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol1_20230502_Flats_1309_Fe_I_6173_fa.FITS

...

reading average flat files:  


  0%|          | 0/1280 [00:00<?, ?it/s]


 /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol1_20230502_Flats_1309_Fe_I_6173_fa.FITS


  0%|          | 0/1280 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def process_flat_fit_parallel(line, channel, mod):
    # Read the settings
    config = configobj.ConfigObj('config.ini')
    ffdir = config['flats']['directory']
    settings = [f for f in os.listdir(ffdir) if 'settings' in f]
    settings = ffdir + os.sep + settings[0]
    settings = configobj.ConfigObj(settings)

    # Other numbers
    iline = 0
    linestr = 'Line_' + str(get_line_num(settings, line, iline))
    nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
    nwav = int(settings[linestr]['NWavePoints'])
    filtstr = settings[linestr]['Filter']
    modstr = settings[linestr]['Polarimeter\\Modulation']
    nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
    nfpc = nmod*nacc*nwav
    nfpw = nmod*nacc
    ncyc = len(os.listdir(ffdir+os.sep+line))//3
    # Read the data
    ffname = config['flats'][line+'/'+channel]
    ff = np.float64(pf.open(ffname)[0].data)
    print(time_now(), 'reading average flat files: ', '\n', ffname)
    Y, X = ff.shape[0:2]
    ffm = ff[:,:,mod,:]
    
    # Fit the data
    fit_params = fit_lines_3d_parallel_poly2(ffm, nparallel=16)
    
    # Save the data
    ffit_name = ffname.replace('_fa', '_mod'+str(mod)+'_ft')
    hdu1 = pf.PrimaryHDU(fit_params)
    hdul = pf.HDUList([hdu1])
    hdul.writeto(ffit_name, overwrite=True)
    hdul.close()
    config['flats'][line+'/'+channel+str(mod)+'_fit'] = ffit_name
    config.write()
    print(time_now(), 'fit data written to: ', '\n', ffit_name)
    return

In [ ]:
config = configobj.ConfigObj('config.ini')
ffdir = config['flats']['directory']
settings = [f for f in os.listdir(ffdir) if 'settings' in f]
settings = ffdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
#
fit_params = []
ffit_name = config['flats'][line+'/pol1_mod0_fit']
fit_params.append(pf.open(ffit_name)[0].data)
ffit_name = config['flats'][line+'/pol1_mod1_fit']
fit_params.append(pf.open(ffit_name)[0].data)
ffit_name = config['flats'][line+'/pol1_mod2_fit']
fit_params.append(pf.open(ffit_name)[0].data)
ffit_name = config['flats'][line+'/pol1_mod3_fit']
fit_params.append(pf.open(ffit_name)[0].data)
ffit_name = config['flats'][line+'/pol2_mod0_fit']
fit_params.append(pf.open(ffit_name)[0].data)
ffit_name = config['flats'][line+'/pol2_mod1_fit']
fit_params.append(pf.open(ffit_name)[0].data)
ffit_name = config['flats'][line+'/pol2_mod2_fit']
fit_params.append(pf.open(ffit_name)[0].data)
ffit_name = config['flats'][line+'/pol2_mod3_fit']
fit_params.append(pf.open(ffit_name)[0].data)
fit_params = np.array(fit_params)

In [ ]:
temp = fit_params[2]
fig, axs = plt.subplots(2,3,figsize=(12,8))
for i, ax in enumerate(axs.flatten()):
    ax.imshow(temp[:,:,i])
fig.tight_layout()

In [ ]:
ipar = 5
plt.imshow(fit_params[0,:,:,ipar]-fit_params[1,:,:,ipar])
plt.colorbar()

In [ ]:
# Sample line fitting with predefined model
ff1name = config['flats'][line+'/pol1']
ff1 = np.float64(pf.open(ff1name)[0].data)
ff2name = config['flats'][line+'/pol2']
ff2 = np.float64(pf.open(ff2name)[0].data)
#
plt.figure()
Y, X, nmod, Z = ff1.shape
y, x  = 375, 30
line_data, wav_ind = ff1[y,x,0], np.arange(Z)
# fit_res = fit_gauss_poly2_parallel(line_data, y, x)
fit1 = fit_gauss_poly2(line_data, y, x)
plt.plot(fit1.best_fit, 'b')
# plt.plot(gauss_poly2(guess_params, wav_ind))
plt.plot(line_data, 'k--')
# plt.plot(line_data)
# plt.plot(res.init_fit)
# fit_res[2]